In [ ]:
import sys

In [ ]:
sys.path.append("../src/")

In [ ]:
%load_ext dotenv
%dotenv ../services/environments/.env.secrets

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
%matplotlib widget

In [ ]:
%load_ext snakeviz

In [ ]:
%matplotlib inline

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
from vxgioq_cfu_counting_ml.api.data_models import Predictions, Region, ShapeAttributes
from vxgioq_cfu_counting_ml.training.dataset_mapper import CFUDatasetMapper
from vxgioq_cfu_counting_ml.training.data_analysis import get_dataset_stats, check_retest_models, compute_kpis, get_data_compute_predictions_save_zip_upload_images_names_predictions_count, construct_model_info, predict_images
from vxgioq_cfu_counting_ml.training.matched_data import MATCHES
from vxgioq_cfu_counting_ml.training.organize_data import get_paths_test_images, get_paths_train_images, count_files, get_paths_positive_unlabelled_images, via_to_detectron2, custom_train_test_split, generate_train_test, gsk_original_to_detectron2, negative_to_detectron2
from vxgioq_cfu_counting_ml.training.trainer import CFUTrainer

from vxgioq_cfu_counting_ml.utils.azure import (
    download_image,
    delete_object_from_blob,
    download_load_models_kpis,
    download_unzip_clean_zip,
    get_cosmos_mongo_collection,
    get_cosmos_mongo_collection_phase2,
    get_file_from_blob,
    list_data_on_blob,
    list_models_on_blob,
    upload_file_to_blob,
    download_unzip_clean_data,
    download_unzip_clean_model,
    write_zip_upload_models_kpis,
    zip_upload_data_to_blob,
    zip_upload_file_folder_to_blob,
    zip_upload_model_to_blob,
    zip_upload_predictions_to_blob,
    ACCOUNT_URL,
    CONTAINER,
    DATASET_DETECTRON2_NAME,
    DATASETS_QUANTILES_NAME,
    MODEL_CONFIG_NAME,
    MODELS_KPIS_NAME,
    MODELS_QUANTILES_NAME,
    MODEL_WEIGHTS_NAME,
    PATH_DATA_ON_DISK,
    PATH_MODELS_ON_DISK,
    PATH_PREDICTIONS_ON_DISK
)
from vxgioq_cfu_counting_ml.utils.conf import Backbone, Device, clean_output_dir_add_cfg, delete_files, delete_folders, prepare_cfu_detectron2_config_from_detectron2_model_zoo, prepeare_cfu_detectron_config_from_existing_model
from vxgioq_cfu_counting_ml.utils.prediction_intervals import pad_arrays, scores_to_distributions, generate_pi_dataset, DataType, train_quantile_regressor, HPSpace, verify_quantiles_predictions, BinningOption, DistributionFunctionType, scores_to_compiled_cdfs
from vxgioq_cfu_counting_ml.utils.convert import enhance_detectron2_dataset, extract_challenging_images, load_scan4000_excel, extract_png_from_scan4000_report, extract_metadata_from_scan4000_report, extract_all_from_scan4000_report
from vxgioq_cfu_counting_ml.utils.custom_datasets import func_dataset, mask_2_polygon
from vxgioq_cfu_counting_ml.utils.display import compare_scan4000_and_dl_images, display_batch, display_pi, display_dataset, display_enhanced_dataset, display_features_importance, display_violin, NormalizeOption, save_examples
from vxgioq_cfu_counting_ml.utils.evaluator import CFUEvaluator
from vxgioq_cfu_counting_ml.utils.loader import load_model, get_best_model_name, get_best_model, load_images
from vxgioq_cfu_counting_ml.utils.metrics import mse, mae, mape, calibration
from vxgioq_cfu_counting_ml.utils.predictor import CFUPredictor
from vxgioq_cfu_counting_ml.utils.types import *


In [ ]:

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())


In [ ]:
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt 
import glob
%matplotlib inline


# New Data collection

In [ ]:
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.database import Database
from urllib.parse import quote_plus
from datetime import datetime,timedelta

In [ ]:
# Azure Cosmos Mongo DB
COSMOS_USER_NAME="us6vxdagioqcfudevcosmosdb"
COSMOS_PASSWORD="dn5UEjj7Svh2aJDBqoBbbB5KkHdYmZ7B0BpPXUQ7Srx2MbhxSFvLejyjggsNJsZABYktRC2b9gBpSDT8F00Jmg=="
COSMOS_HOST="us6vxdagioqcfudevcosmosdb.mongo.cosmos.azure.com"
COSMOS_APP_NAME="us6vxdagioqcfudevcosmosdb"
COSMOS_PORT = "10255"
COSMOS_DATABASE_NAME = "cfu_database"
COSMOS_COLLECTION_NAME = "cfu_counting_collection"
COSMOS_COLLECTION_NAME_PHASE2 = "cfu_counting_collection_phase2"

In [ ]:
CONNECTION_URI = f"mongodb://{quote_plus(COSMOS_USER_NAME)}:{quote_plus(COSMOS_PASSWORD)}@{COSMOS_HOST}:{COSMOS_PORT}/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@{COSMOS_APP_NAME}@"

In [ ]:
def get_cosmos_mongo_client() -> MongoClient:
    """Authenticate and create CosmosClient."""
    return MongoClient(CONNECTION_URI)

def get_cosmos_mongo_db() -> Database:
    """Get Azure cosmos mongo database."""
    client = get_cosmos_mongo_client()
    return client[COSMOS_DATABASE_NAME]

def get_cosmos_mongo_collection() -> Collection:
    """Get Azure cosmos mongo collection."""
    db = get_cosmos_mongo_db()
    return db[COSMOS_COLLECTION_NAME]

def get_cosmos_mongo_collection_phase2() -> Collection:
    """Get Azure cosmos mongo collection."""
    db = get_cosmos_mongo_db()
    return db[COSMOS_COLLECTION_NAME_PHASE2]

In [ ]:
#downloading model Kpi json frpm blob
def get_segmentation_model_kpi():
    ret=get_file_from_blob('https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/model_kpi_segmentation.json').read()
    ret=json.loads(ret.decode('utf-8'))
    return ret  
model_kpi=get_segmentation_model_kpi()

In [ ]:
#loading model kpi
model_kpi


In [ ]:
list_models_on_blob()

In [ ]:
dishes_collection2 = get_cosmos_mongo_collection_phase2()

In [ ]:
last_month = True #if we need images from the past 30 days
no_of_images = 1000 #the number of images selected
days = 30 #number of days
no_of_days = timedelta(days)

In [ ]:
if(last_month):
    #selects the start date 30 days before the current date
    ending_date = datetime.now()
    starting_date = ending_date - no_of_days



else:
    #for a particular timespan
    a=[]
    for i in (model_kpi['models'][-1]['time_stamp'].split(" ")[0]).split('-'):
        a.append(int(i))
    starting_date=datetime(a[0],a[1],a[2])
    #starting_date = datetime(int(model_kpi['models'][-1]['time_stamp'])) # collecting data based on last trained model date.
    ending_date =datetime.now()
start_date = starting_date.strftime('%Y-%m-%dT%H:%M:%S.%sZ')
end_date = ending_date.strftime('%Y-%m-%dT%H:%M:%S.%sZ')

In [ ]:
dishes = list(dishes_collection2.find({ "$and": [ { "is_validated":1 }, { 'corrections.image.validated_datetime': {"$gte":start_date, "$lte":end_date} } ] }).limit(no_of_images))

In [ ]:
Print("no of images collected :",len(dishes))


# Deleting the previously created fodlers on blob storage

In [ ]:
import shutil

shutil.rmtree(F'{PATH_DATA_ON_DISK}/data_segmentation')

# Creating folders on DISK to store Images and Json

In [ ]:
data_path=PATH_DATA_ON_DISK
directory="data_segmentation/train"
path = os.path.join(data_path, directory) 
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % directory)
except OSError as error:
    print("Directory '%s' can not be created" % directory)
#os.makedirs(path)
directory="data_segmentation/validated_json"
path = os.path.join(data_path, directory) 
#os.makedirs(path)
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % directory)
except OSError as error:
    print("Directory '%s' can not be created" % directory)

In [ ]:
#path to save the validated json and corresponding images
image_path=PATH_DATA_ON_DISK+'/data_segmentation/train/'           #image path on Disk
json_path=PATH_DATA_ON_DISK+'/data_segmentation/validated_json/'  #json path on Disk


In [ ]:
for i in dishes: 
    i['_id']=str(i['_id'])
    i['scanned_datetime']=str(i['scanned_datetime'])
    i['predictions']['image']['predictions_datetime']=str(i['predictions']['image']['predictions_datetime'])
   
    data=download_image(i['image_original_url'])
    img = Image.fromarray(data, 'RGB')
    #plt.imshow(img)
    img.save(image_path+i['image_original_url'].split('/')[-1])
    json_object=json.dumps(i,indent=4)
    with open(json_path+i['sample_id']+'.json','w') as outfile:
        outfile.write(json_object)

# converting the json into coco format

In [ ]:
d = dict()
d['images'] = list() 
d['categories']=[
   
        {
            "supercategory": "mold",
            "id": 1,
            "name": "mold"
        }
    ]
d['annotations'] =list()
import json
import numpy as np
import glob

path = PATH_DATA_ON_DISK+'/data_segmentation/validated_json/*.*'
l=[]
for filename in glob.glob(path):
    l.append(filename)

counter = 1

for r in range(len(l)):
    with open(l[r]) as x:
        f = json.load(x)
        data = download_image(f['image_original_url'])
        img = Image.fromarray(data, 'RGB')
        w, h = img.size
        
        #filtering only mold annortation by category id = 1
        for i in range(len(f['corrections'][-1]['image']['regions'])):
            if(f['corrections'][-1]['image']['regions'][i]['shape_attributes']['category_id']==1) and (f['corrections'][-1]['image']['regions'][i]['shape_attributes']['name']=='polygon'):
                
                segm = []
                x = f['corrections'][-1]['image']['regions'][i]['shape_attributes']['all_points_x']
                y = f['corrections'][-1]['image']['regions'][i]['shape_attributes']['all_points_y']
                for i in range(len(x)):
                    segm.append(x[i])
                    segm.append(y[i])
 
                z =list(zip(x,y))
                contour = np.array(z)
                x = contour[:, 0]
                y = contour[:, 1]
                ar = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
                
                minx = int(min(x))
                maxx = int(max(x))
                miny = int(min(y))
                maxy = int(max(y))

                bbox = [minx , miny, maxx, maxy]
                d['annotations'].append({'segmentation':[segm], 
                                  'iscrowd':0, 
                                  'area':ar, 
                                  'image_id':int(r), 
                                  'bbox':bbox, 
                                    'category_id':1, 
                                  'id':int(counter)})
                counter = counter + 1
                
                d['images'].append({
                    "height": h,  
                    "width": w,    
                    "id": int(r),
                    "file_name": f['sample_id']+'.png'
                    })

In [ ]:
with open(PATH_DATA_ON_DISK+'/data_segmentation/train.json','w') as dw: #creating train.json file and dumping the coco annotation
    json.dump(d,dw)

# Data Register

In [ ]:
from detectron2.data.datasets import register_coco_instances

for d in ["train"]:
    
    register_coco_instances(f"coco_{d}", {}, f"{PATH_DATA_ON_DISK}/data_segmentation/{d}.json",
                           f"{PATH_DATA_ON_DISK}/data_segmentation/{d}")

In [ ]:
import random
import matplotlib.pyplot as plt
from detectron2.data import DatasetCatalog, MetadataCatalog

dataset_dicts = DatasetCatalog.get("coco_train")
coco_metadata = MetadataCatalog.get("coco_train")

#for d in random.sample(dataset_dicts, 3):
for d in dataset_dicts:
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    v = Visualizer(img[:, :, ::-1], metadata=coco_metadata, scale=0.5)
    v = v.draw_dataset_dict(d)
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("coco_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.001     # do not decay learning rate
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Model training

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)  #Using deafult trainer 
trainer.resume_or_load(resume=False)
trainer.train()

# Downloading the previous model from blob

In [ ]:
old_model_path=model_kpi['models'][-1]['model_path']
old_model_name=model_kpi['models'][-1]['model_name']

In [ ]:
old_model_path

In [ ]:
old_model_name

In [ ]:
old_model_path.split("/")[-1]

In [ ]:
download_unzip_clean_model(old_model_path.split("/")[-1], overwrite=False)

# Save the new trained  model

In [ ]:
model_save_name = 'detectron'+str(int(old_model_name.split('_')[0][-1])+1)+'_mold_model.pth'  #model version 

In [ ]:
model_save_name

In [ ]:
os.makedirs("segmentation_model") #creating new folder to store newly trained dmodel

In [ ]:
model_save_name = 'detectron'+str(int(old_model_name.split('_')[0][-1])+1)+'_mold_model.pth'
new_model_path = F"segmentation_model/{model_save_name}" 
torch.save(trainer.model.state_dict(), new_model_path)

# Prediction with old model

In [ ]:
#prediction of new test images with the old model
image_path=PATH_DATA_ON_DISK+'/data_segmentation/train/'

for file_name in glob.glob(image_path+'*.*'):
    
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # set threshold for this model



    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    

    cfg.MODEL.WEIGHTS =PATH_MODELS_ON_DISK+"/"+old_model_name #weights of previous model
    
    im = cv2.imread(file_name)
    predictor = DefaultPredictor(cfg)
    outputs = predictor(im)
    print(outputs)
    


# Prediction with new model

In [ ]:
#prediction of new test images with the new model
image_path=PATH_DATA_ON_DISK+'/data_segmentation/train/'

for file_name in glob.glob(image_path+'*.*'):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # set threshold for this model



    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    

    cfg.MODEL.WEIGHTS =PATH_MODELS_ON_DISK+"/"+model_save_name #Weights of newly trained model
   
    im = cv2.imread(file_name)
    predictor_new = DefaultPredictor(cfg)
    outputs_new = predictor_new(im)
    print(outputs_new)
    

# calculating AP(Average Precision) with old model

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("coco_train")
val_loader = build_detection_test_loader(cfg, "coco_train")
result = inference_on_dataset(predictor.model, val_loader, evaluator) #predictor old model
print(result)
AP_prev = []
AP_prev.append(int(result['segm']['AP']))

print(AP_prev)


# calculating AP(Average Precision) with new model

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("coco_train")
val_loader = build_detection_test_loader(cfg, "coco_train")
result = inference_on_dataset(predictor_new.model, val_loader, evaluator) #predictor new model
print(result)
AP_new = []
AP_new.append(int(result['segm']['AP']))

print(AP_new)

# Uploading model and Updating model kpi only if the new trained model AP is greater than exisitng model 

In [ ]:
new_model_path = model_save_name.split(".")[0]
new_model_path

In [ ]:


#updating the kpi only if the new model AP is greater then previous model AP
if AP_new > AP_prev:
    
    #Uploading model to blob
    zip_upload_file_folder_to_blob("segmentation_model/",
                                   F"https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/{new_model_path}.zip",overwrite =False)
    model_kpi['models'].append({
        'model_name':model_save_name,
        'model_path':F"https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/{new_model_path}.zip",
        'AP':AP_new[0],
        'time_stamp':str(datetime.now())
    })
        
    
    with open("model_kpi_segmentation.json",'w') as f:
        json.dump(model_kpi,f)
    
    #uploading the Kpi json to blob
    upload_file_to_blob("model_kpi_segmentation.json",
                        url='https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/model_kpi_segmentation.json',overwrite=True)
        
else:
    print("Previous model is performing better than new model")

    
        
    